## OpenAI's Agents SDK

In [1]:
#!pip install openai-agents

In [ ]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or \
  getpass("Enter your OpenAI API key: ")

In [26]:
from agents import Agent, Runner

agent = Agent(
    name="Assistant",
    instructions="You're a helpful assistant",
    model="gpt-4o-mini",
)

## Running our Agent

In [27]:
result = await Runner.run(
    starting_agent=agent,
    input="짧은 이야기를 만들어줄래."
)
result.final_output

'물론이죠! 여기 짧은 이야기 하나 있어요.\n\n한 작은 마을에 노란색 꽃밭이 있었습니다. 마을 사람들은 그 꽃밭을 특별히 아끼고 사랑했죠. 그러던 어느 날, 꽃밭에 갑자기 파란색 꽃 한 송이가 피었습니다. 사람들은 처음에는 이상하게 여겼지만, 꽃은 점점 아름다움을 더해갔습니다.\n\n아이들은 파란색 꽃을 보며 상상의 나래를 펼쳤고, 어른들은 그 꽃의 특별함을 깨닫게 되었습니다. 결국, 마을 사람들은 서로의 다름을 인정하고 존중하는 법을 배우게 되었고, 파란색 꽃은 마을의 새로운 상징이 되었습니다.\n\n이렇게 작은 꽃 한 송이가 마을을 하나로 묶어주었다는 이야기였습니다.'

In [28]:
response = Runner.run_streamed(
    starting_agent=agent,
    input="여기, 안녕하세요."
)
async for event in response.stream_events():
    print(event)

AgentUpdatedStreamEvent(new_agent=Agent(name='Assistant', instructions="You're a helpful assistant", handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[], input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again'), type='agent_updated_stream_event')
RawResponsesStreamEvent(data=ResponseCreatedEvent(response=Response(id='resp_67de62de1c5c81918c3905e8939b588f0194e2e59f1496c5', created_at=1742627550.0, error=None, incomplete_details=None, instructions="You're a helpful assistant", metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, max_output_tokens=None, previous_response_id=None, reasoning=Reasoning(effort=None, generate_summary=None),

In [29]:
from openai.types.responses import ResponseTextDeltaEvent

# we do need to reinitialize our runner before re-executing
response = Runner.run_streamed(
    starting_agent=agent,
    input="짧은 이야기를 들려주세요."
)

async for event in response.stream_events():
    if event.type == "raw_response_event" and \
        isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

한 작은 마을에 한 노인이 살고 있었습니다. 그는 매일 아침마다 마을의 넓은 공원에 나가서 나무와 꽃들에게 물을 주며 시간을 보냈습니다. 사람들은 그를 "자연의 친구"라고 불렀습니다.

어느 날, 한 아이가 노인에게 다가와 물었습니다. "왜 이렇게 열심히 나무에 물을 주시나요?" 노인은 미소를 지으며 대답했습니다. "우리가 언제까지나 함께 할 수는 없지만, 나무와 꽃은 그 자리를 지켜줄 거란다. 우리가 사랑한 그 자리에서 또 다른 생명들이 자랄 수 있도록."

아이의 눈이 밝아지며 말했습니다. "그러면 나도 사랑하는 나무를 키우고 싶어요!" 노인은 고개를 끄덕이며 작은 나무를 하나 주었고, 아이는 그 나무를 정성껏 가꾸기로 다짐했습니다.

세월이 흐른 후, 그 아이는 자란 나무 아래에서 친구들과 함께 놀며 행복한 시간을 보냈습니다. 그리고 노인이 정성스럽게 가꾸었던 공원은 여전히 아름다웠습니다. 누군가의 사랑은 결국 다른 이를 통해 세대를 넘어 계속 이어지는 법이니까요.

## Tools

In [30]:
from agents import function_tool

@function_tool
def multiply(x: float, y: float) -> float:
    """정확한 답을 제공하기 위해 `x`와 `y`를 곱합니다.
    답을 제공해줘."""
    return x*y

In [31]:
agent = Agent(
    name="Assistant",
    instructions=(
        "당신은 유용한 도우미입니다, 항상 "
        "가능하면 항상 제공된 도구를 사용하세요. 하지 마세요 "
        "자신의 지식에 지나치게 의존하지 말고 대신 "
        "도구를 사용하여 질문에 답하세요."
    ),
    model="gpt-4o-mini",
    tools=[multiply]  # note that we expect a list of tools
)

In [32]:
response = Runner.run_streamed(
    starting_agent=agent,
    input="10.14에 77.92를 곱한 값은 무엇인가요?"
)

async for event in response.stream_events():
    print(event)

AgentUpdatedStreamEvent(new_agent=Agent(name='Assistant', instructions='당신은 유용한 도우미입니다, 항상 가능하면 항상 제공된 도구를 사용하세요. 하지 마세요 자신의 지식에 지나치게 의존하지 말고 대신 도구를 사용하여 질문에 답하세요.', handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[FunctionTool(name='multiply', description='정확한 답을 제공하기 위해 `x`와 `y`를 곱합니다.\n답을 제공해줘.', params_json_schema={'properties': {'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}, 'required': ['x', 'y'], 'title': 'multiply_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7fc5a5f0fe20>, strict_json_schema=True)], input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again'), type='agent_updated_stream_eve

In [33]:
from openai.types.responses import (
    ResponseFunctionCallArgumentsDeltaEvent,  # tool call streaming
    ResponseCreatedEvent,  # start of new event like tool call or final answer
)

response = Runner.run_streamed(
    starting_agent=agent,
    input="10.14에 77.92를 곱한 값은 무엇인가요?"
)

async for event in response.stream_events():
    if event.type == "raw_response_event":
        if isinstance(event.data, ResponseFunctionCallArgumentsDeltaEvent):
            # this is streamed parameters for our tool call
            print(event.data.delta, end="", flush=True)
        elif isinstance(event.data, ResponseTextDeltaEvent):
            # this is streamed final answer tokens
            print(event.data.delta, end="", flush=True)
    elif event.type == "agent_updated_stream_event":
        # this tells us which agent is currently in use
        print(f"> Current Agent: {event.new_agent.name}")
    elif event.type == "run_item_stream_event":
        # these are events containing info that we'd typically
        # stream out to a user or some downstream process
        if event.name == "tool_called":
            # this is the collection of our _full_ tool call after our tool
            # tokens have all been streamed
            print()
            print(f"> Tool Called, name: {event.item.raw_item.name}")
            print(f"> Tool Called, args: {event.item.raw_item.arguments}")
        elif event.name == "tool_output":
            # this is the response from our tool execution
            print(f"> Tool Output: {event.item.raw_item['output']}")

> Current Agent: Assistant
{"x":10.14,"y":77.92}
> Tool Called, name: multiply
> Tool Called, args: {"x":10.14,"y":77.92}
> Tool Output: 790.1088000000001
10.14에 77.92를 곱한 값은 약 790.11입니다.

## Guardrails

In [34]:
from pydantic import BaseModel

# define structure of output for any guardrail agents
class GuardrailOutput(BaseModel):
    is_triggered: bool
    reasoning: str

# define an agent that checks if user is asking about political opinions
politics_agent = Agent(
    name="Politics check",
    instructions="사용자가 정치적 의견을 묻는지 확인하세요.",
    output_type=GuardrailOutput,
)

In [35]:
query = "미국의 트럼프 2기에 대해 어떻게 생각하시나요? 한글로 대답"

result = await Runner.run(
    starting_agent=politics_agent, 
    input=query
)
result

RunResult(input='미국의 트럼프 2기에 대해 어떻게 생각하시나요? 한글로 대답', new_items=[MessageOutputItem(agent=Agent(name='Politics check', instructions='사용자가 정치적 의견을 묻는지 확인하세요.', handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[], input_guardrails=[], output_guardrails=[], output_type=<class '__main__.GuardrailOutput'>, hooks=None, tool_use_behavior='run_llm_again'), raw_item=ResponseOutputMessage(id='msg_67de62eb39ec8191b60f66f60caa593e07c8bf989960b4db', content=[ResponseOutputText(annotations=[], text='{"is_triggered":true,"reasoning":"이 질문은 특정 정치적 인물과 그의 재임에 대한 의견을 묻고 있습니다. 이는 정치적 의견 요청에 해당합니다."}', type='output_text')], role='assistant', status='completed', type='message'), type='message_output_item')], raw_responses=[ModelResponse(output=[ResponseOutputMessage(id='msg_67de62eb39ec8191b60f66f60caa593e07c8bf989960

In [36]:
result.final_output

GuardrailOutput(is_triggered=True, reasoning='이 질문은 특정 정치적 인물과 그의 재임에 대한 의견을 묻고 있습니다. 이는 정치적 의견 요청에 해당합니다.')

In [37]:
from agents import (
    GuardrailFunctionOutput,
    RunContextWrapper,
    input_guardrail
)

# this is the guardrail function that returns GuardrailFunctionOutput object
@input_guardrail
async def politics_guardrail(
    ctx: RunContextWrapper[None],
    agent: Agent,
    input: str,
) -> GuardrailFunctionOutput:
    # run agent to check if guardrail is triggered
    response = await Runner.run(starting_agent=politics_agent, input=input)
    # format response into GuardrailFunctionOutput
    return GuardrailFunctionOutput(
        output_info=response.final_output,
        tripwire_triggered=response.final_output.is_triggered,
    )

In [39]:
agent = Agent(
    name="Assistant",
    instructions=(
        "당신은 유용한 assistants입니다."
        "가능하면 제공된 도구를 사용하세요."
        "자신의 지식에 지나치게 의존하지 말고 대신."
        "도구를 사용하여 질문에 답하세요."
    ),
    model="gpt-4o-mini",
    tools=[multiply],
    input_guardrails=[politics_guardrail],  # note this is a list of guardrails
)

Now let's run it! We'll stick with `Runner.run` for the sake of brevity:
이제 실행해 봅시다! 간결함을 위해 `Runner.run`을 계속 사용하겠습니다:

In [40]:
result = await Runner.run(
    starting_agent=agent,
    input="10.14에 77.92를 곱한 값은 무엇인가요?"
)
result.final_output

'10.14에 77.92를 곱한 값은 790.11입니다.'

Let's see if our guardrail will trigger:

In [41]:
result = await Runner.run(
    starting_agent=agent,
    input="미국의 트럼프 2기에 대해 어떻게 생각하시나요? 한글로 대답?"
)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

## Conversational Agents

In [42]:
result = await Runner.run(
    starting_agent=agent,
    input="10.645라는 숫자를 기억해 주세요."
)
result.final_output

'그 숫자를 기억했습니다: 10.645. 도움이 필요하시면 말씀해 주세요!'

In [43]:
result.to_input_list()

[{'content': '10.645라는 숫자를 기억해 주세요.', 'role': 'user'},
 {'id': 'msg_67de63916e0481918f35f3e82ea01e64098f9ee6d31dd742',
  'content': [{'annotations': [],
    'text': '그 숫자를 기억했습니다: 10.645. 도움이 필요하시면 말씀해 주세요!',
    'type': 'output_text'}],
  'role': 'assistant',
  'status': 'completed',
  'type': 'message'}]

In [44]:
result = await Runner.run(
    starting_agent=agent,
    input=result.to_input_list() + [
        {"role": "user", "content": "마지막 숫자에 113.932를 곱합니다."}
    ]
)
result.final_output

'10.645와 113.932를 곱한 결과는 1212.80614입니다. 더 필요하신 정보가 있으면 말씀해 주세요!'